In [1]:
import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

# Paths
input_root = "Resized_IMG_CLASSES"
output_root = "normalized_images"

# Models and their specific normalization transforms
model_transforms = {
    "efficientnet": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225]),
    "vgg": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
    "densenet": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
    "resnet": transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225]),
    "mobilenetv2": None  # Only /255 scaling, handled separately
}

# Base transformation (resize skipped)
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# Loop through model types
for model_name, norm_transform in model_transforms.items():
    print(f"Processing for {model_name}...")

    model_output_dir = os.path.join(output_root, model_name)
    os.makedirs(model_output_dir, exist_ok=True)

    # Go through each class folder
    for class_name in os.listdir(input_root):
        class_input_dir = os.path.join(input_root, class_name)
        class_output_dir = os.path.join(model_output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)

        # Loop through each image in the class folder
        for img_name in tqdm(os.listdir(class_input_dir), desc=f"{model_name} - {class_name}"):
            img_path = os.path.join(class_input_dir, img_name)
            try:
                image = Image.open(img_path).convert("RGB")
                tensor = to_tensor(image)

                if norm_transform:
                    tensor = norm_transform(tensor)  # Standard normalization
                else:
                    # MobileNetV2: only scaling to [0, 1], already done by ToTensor
                    pass

                # Convert back to PIL image (scaled for saving)
                norm_image = to_pil(tensor)
                norm_image.save(os.path.join(class_output_dir, img_name), format='JPEG')

            except Exception as e:
                print(f"Error processing {img_path}: {e}")


Processing for efficientnet...


efficientnet - 7. Viral Infections: 100%|██████████| 1000/1000 [00:07<00:00, 132.54it/s]


Processing for vgg...


vgg - 7. Viral Infections: 100%|██████████| 1000/1000 [00:07<00:00, 128.78it/s]


Processing for densenet...


densenet - 7. Viral Infections: 100%|██████████| 1000/1000 [00:07<00:00, 132.12it/s]


Processing for resnet...


resnet - 7. Viral Infections: 100%|██████████| 1000/1000 [00:07<00:00, 125.59it/s]


Processing for mobilenetv2...


mobilenetv2 - 7. Viral Infections: 100%|██████████| 1000/1000 [00:07<00:00, 136.31it/s]
